# End-to-End Freebase QA

Use the below line to import modules from the root of the repository.

Reference: https://stackoverflow.com/questions/34976803/sys-path-different-in-jupyter-and-python-how-to-import-own-modules-in-jupyter

In [57]:
import sys
sys.path.insert(0, '../../')

In [58]:
from scripts.utils.connect import get_connection 

connection = get_connection()
cursor = connection.cursor()

In [59]:
from scripts.utils.simple_qa import load_simple_qa 

df_dev, = load_simple_qa(dev=True)
df_dev[:5]

,subject,relation,object,question
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...


In [47]:
from functools import partial
from tqdm import tqdm_notebook
from numpy import nan
from IPython.display import display, Markdown
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer
import unicodedata
import re


tokenizer = TweetTokenizer()
stemmer = SnowballStemmer('english', ignore_stopwords=True)

def stem_phrase(s):
    s = s.lower()
    if len(s) >= 1and s[-1] in ['?', '.', '!']: # Remove end of sentence punctuation
        s = s[0:-1]
    # https://stackoverflow.com/questions/34714162/preventing-splitting-at-apostrophies-when-tokenizing-words-using-nltk
    s = [word.strip() for word in tokenizer.tokenize(s)]
    s = [stemmer.stem(word) for word in s]
    return ' '.join(s)
    
def get_name_in_question(question, names):
    """ Get the longest name in the question """
    for i, name in enumerate(names):
        if name.lower() in question:
            return i
    return nan

def add_subject_name(df, print_=False):
    n_failed_no_subject_reference = 0
    n_failed_no_alias = 0
    subject_names = []
    print_data = []
    
    for index, row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
        cursor.execute("SELECT alias FROM fb_name WHERE mid=%s", (row['subject'], ))
        rows = cursor.fetchall()
        if len(rows) == 0:
            n_failed_no_alias += 1
            if print_:
                print('Subject MID (%s) does not have aliases.' % row['subject'])
            subject_names.append(nan)
            df.loc[index, 'subject_name'] = nan
            continue
        names = [row[0].strip() for row in rows]
        names = sorted([name for name in names], key=lambda k: len(k), reverse=True)
        stemmed_names = [stem_phrase(name) for name in names]
        stemmed_question = stem_phrase(row['question'])
        name_index = get_name_in_question(stemmed_question, stemmed_names)
        if name_index is nan:
            n_failed_no_subject_reference += 1
            print_data.append({
                'Question Stemmed': stemmed_question,
                'Question': row['question'],
                'Subject': row['subject'],
                'Names Stemmed': stemmed_names,
                'Names': names,
            })
            subject_names.append(nan)
        else:
            subject_names.append(names[name_index])
        
    df['subject_name'] = pd.Series(subject_names, index=df.index)

    if print_:
        display(pd.DataFrame(print_data)[:50])
        display(Markdown(
            '### Numbers' + 
            '\n%f%% [%d of %d] questions do not reference subject' %
            ((n_failed_no_subject_reference / df.shape[0]) * 100,
             n_failed_no_subject_reference, df.shape[0]) +
            '\n\n%f%% [%d of %d] subject mids do not have aliases' %
            ((n_failed_no_alias / df.shape[0]) * 100, n_failed_no_alias, df.shape[0])
        ))

add_subject_name(df_dev, print_=True)

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))

Subject MID (0yzv6q0) does not have aliases.
Subject MID (0k571hg) does not have aliases.
Subject MID (02z6y3p) does not have aliases.
Subject MID (08vqg8n) does not have aliases.
Subject MID (045p9_r) does not have aliases.
Subject MID (03czw5g) does not have aliases.
Subject MID (04j2m25) does not have aliases.
Subject MID (0bvrjh4) does not have aliases.
Subject MID (08c_49) does not have aliases.
Subject MID (0h4867) does not have aliases.
Subject MID (02qplxv) does not have aliases.
Subject MID (0kz5hyk) does not have aliases.
Subject MID (07wk3m) does not have aliases.
Subject MID (01jyqwk) does not have aliases.
Subject MID (0_s2kgd) does not have aliases.
Subject MID (02vk_bv) does not have aliases.
Subject MID (03f86d) does not have aliases.
Subject MID (0kyvhwk) does not have aliases.
Subject MID (0l8d74) does not have aliases.
Subject MID (04jnfd) does not have aliases.
Subject MID (04zfl8) does not have aliases.
Subject MID (0z5y43n) does not have aliases.
Subject MID (040b

,Names,Names Stemmed,Question,Question Stemmed,Subject
0,"[megan leigh romero, megan romero]","[megan leigh romero, megan romero]",What track is featured on the salisbury release,what track is featur on the salisburi releas,098j5p7
1,[datskat],[datskat],What composer created?,what compos creat,0zhwd1v
2,[ron warner],[ron warner],Who's a linebacker,who a lineback,03m9zc3
3,"[sabrina fredrica washington, washington, sabr...","[sabrina fredrica washington, washington , sab...",Which label is sabrinawmusic signed to?,which label is sabrinawmus sign to,0bnsl7
4,"[wagoner county / tulsa city, tulsa, oklahoma,...","[wagon counti / tulsa citi, tulsa , oklahoma, ...",What newspaper circulates in the town of kearny,what newspap circul in the town of kearni,013kcv
5,[nguyễn văn toàn],[nguyễn văn toàn],Where is the place of birth of nguyen van toan,where is the place of birth of nguyen van toan,02vnwfd
6,"[capital lights, aftereight]","[capit light, aftereight]",what types of music is played by capitallights,what type of music is play by capitallight,02pvx1q
7,"[35 mm film, 35mm film]","[35 mm film, 35mm film]",what is an example of a film on 35mm,what is an exampl of a film on 35mm,0cj16
8,[studio album],[studio album],What is the name of the award winning album by...,what is the name of the award win album by lin...,06vw6v
9,"[music of latin america, latin music]","[music of latin america, latin music]",What is an artist associated with latin americ...,what is an artist associ with latin american m...,0g293


### Numbers
2.268326% [246 of 10845] questions do not reference subject

0.461042% [50 of 10845] subject mids do not have aliases

In [50]:
RELATION_CLASSIFIER = '../../results/0626.11-05_08:38:02.relation_classifier/11m_05d_08h_45m_57s.pt'
OBJECT_RECOGNITION = '../../results/0605.11-05_09:35:18.object_recognition/11m_05d_09h_45m_22s.pt'

import math
import pprint
import re
import unicodedata

from nltk.tokenize.treebank import TreebankWordTokenizer

from lib.checkpoint import Checkpoint

pretty_printer = pprint.PrettyPrinter(indent=2)
tokenizer = TreebankWordTokenizer() # Same tokenizer used during training
    
def remove_accents(text):
    # https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string
    nkfd_form = unicodedata.normalize('NFKD', text)
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])

def preprocess_question(s):
    s = s.replace("'s", '')
    s = remove_accents(s)
    s = s.split()
    s = [w.strip().lower() for w in s]
    s = [w for w in s if len(w) != 0] # Remove Empty
    return ' '.join(s)

def get_predict(checkpoint_path):
    """ Get a predict callable for a model """
    checkpoint = Checkpoint(checkpoint_path=checkpoint_path)
    return checkpoint.predict

relation_classifier_predict = get_predict(RELATION_CLASSIFIER)
object_recognition_predict = get_predict(OBJECT_RECOGNITION)

def get_object(question):
    """ Given a question return the object in the question using `OBJECT_CHECKPOINT` model. """
    question = preprocess_question(question)
    marks, confidence = object_recognition_predict(question)
    marks = marks.split()[:-1]
    question = tokenizer.tokenize(question)
    if len(question) != len(marks):
        print('Warning: Marks and Question does not match up. %s, %s.' % (marks, question))
    entity = []
    for marker, word in zip(marks, question):
        if marker == 'e':
            entity.append(word)
    return ' '.join(entity), sum(confidence) / len(confidence)

def get_relation(question, top_k=3):
    """ 
    Given a question return the predicate in the question using `RELATION_CLASSIFIER` model.
    
    Args:
        question (str)
    Returns:
        list of predicates and their confidence
    """
    question = preprocess_question(question)
    predicted = list(relation_classifier_predict(question, top_k=top_k))
    if top_k == 1:
            predicted[1] = sum(predicted[1]) / len(predicted[1]) 
            return [predicted]
    return [(class_, sum(confidence) / len(confidence)) for class_, confidence in predicted]

# To test this cell
print(get_object('Where was Obama born?'))
print(get_relation('Where was Obama born?', 1))

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'lib.nn.relation_classifier.RelationClassifier' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


('obama', 0.9997887267225003)
[['www.freebase.com/people/person/place_of_birth', 1.0]]


/usr/local/lib/python3.5/dist-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'lib.nn.seq_to_seq.SeqToSeq' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'lib.nn.seq_encoder.SeqEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'lib.nn.seq_decoder.SeqDecoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Mod

In [51]:
predicted_subject_names = []
predicted_subject_names_confidence = []

for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    predicted_subect_name, confidence = get_object(row['question'])
    predicted_subject_names.append(predicted_subect_name)
    predicted_subject_names_confidence.append(confidence)

df_dev['predicted_subject_name'] = pd.Series(predicted_subject_names, index=df_dev.index)
df_dev['predicted_subject_name_confidence'] = pd.Series(predicted_subject_names_confidence, index=df_dev.index)
    
df_dev[:50]

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))

Decoder has not predicted EOS in 1000 iterations. Breaking.


,subject,relation,object,question,subject_name,predicted_subject_name,predicted_subject_name_confidence
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...,trump ocean club international hotel and tower,trump ocean club international hotel and tower,0.994040
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born,sasha vujačić,sasha vujacic,0.999662
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in,dead combo,dead combo,0.995047
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?,wiebke von carolsfeld,wiebke von carolsfeld,0.998660
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...,music for stock exchange,stock exchange,0.982731
5,02wcf0f,location/location/containedby,09c7w0,Where is adler school of professional psycholo...,adler school of professional psychology,adler school of professional psychology,0.995717
6,0c1mfxz,people/person/place_of_birth,01m1_t,where was john miltern born,john miltern,john miltern,0.999866
7,02ws83,sports/sports_team/location,080h2,what city is vancouver millionaires from?,vancouver millionaires,vancouver millionaires,0.999528
8,02p_vkx,people/person/profession,04gc2,What was Seymour Parker Gilbert's profession?,seymour parker gilbert,seymour parker gilbert,0.998771
9,0k0057k,astronomy/orbital_relationship/orbits,06m_p,what does (12385) 1994 uo orbit,(12385) 1994 uo,( 12385 ),0.996176


In [55]:
correct = 0
for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    if row['subject_name'] and row['predicted_subject_name'] == row['subject_name']:
        correct += 1
    elif row['subject_name']:
        print(row['question'])
        print(row['predicted_subject_name'])
        print(row['subject_name'])
        print()
print('Accuracy:', correct / df_dev.shape[0])

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))

where was sasha vujačić born
sasha vujacic
sasha vujačić

what country was music for stock exchange  released in
stock exchange
music for stock exchange

what does (12385) 1994 uo orbit
( 12385 )
(12385) 1994 uo

in what french city did antoine de févin die 
antoine de fevin
antoine de févin

What is the language of the invaders?
invaders
the invaders

Who is the producer on evangelion: 3.0 you can (not) redo?
evangelion : 3.0 you can (
evangelion: 3.0 you can (not) redo

what medium was smooth 'n swingin' released on?
smooth 'n swingin
smooth 'n swingin'

What was bill hosket, jr.'s position 
bill hosket ,
bill hosket, jr.

what is the name for meccas canonical version
meccas
mecca

what is a live action action comedy film produce by spyglass entertainment
action comedy
spyglass entertainment

who wrote gulliver's travels
gulliver travels
gulliver's travels

What type of harp is traditionally found in ireland
ireland
harp

what mountain range does the mountain bisaurin belong to
mount

What is rubén quevedo known as?
ruben quevedo
rubén quevedo

what kinds of movie is that'll be the day in netflix
that 'll be the
that'll be the day

what country does kalua‘aha belong to
kalua ‘ aha
kalua‘aha

What's an album by the band portal
portal
album

what genre is the film the girl in the picture under?
girl in the picture
the girl in the picture

What is sébastien menestrier's gender?
sebastien menestrier
sébastien menestrier

what album is by the dixie chicks?
dixie chicks
the dixie chicks

What position does the soccer athlete joaquín enrique valerio play
enrique valerio
joaquín enrique valerio

what kinds of music is cosmo's factory
cosmo factory
cosmo's factory

who sang the song 1club.fm: tha flava (uncensored)
1club.fm : tha flava
1club.fm: tha flava (uncensored)

what is a farce film
farce film
farce

what is jacques gréber's city of birth?
jacques greber
jacques gréber

what nationality is johan löfstedt
johan lofstedt
johan löfstedt

what type of album is the day you

three musketeers
the three musketeers

When did rugby union stop being an olympic sport?
rugby union stop
rugby union

in what language was the im rhein, im schönen strome composition done
im rhein , im schonen
im rhein, im schönen strome

What series includes the episode spring wipeout: dirty laundry?
spring wipeout : dirty
spring wipeout: dirty laundry

what type of album is breaking bad
breaking bad
nan

Which TV series is lost - season 1 a series in?
lost - season
lost - season 1

What sort of movie is the dark hour
dark hour
the dark hour

which language is spoken in kracker jackd
kracker jackd
nan

which country is of the essence from
essence
of the essence

what is the main language in the movie the stolen loaf
stolen loaf
the stolen loaf

What's an example of a piece of art made using oil paint
oil paint
oil paints

What's a greatest hits collection featuring the the marvelettes
marvelettes
the marvelettes

What's an album by michael nyman
michael nyman
michael nymans

which fe

What genre of music is what makes a man start fires??
what makes a man start fires
what makes a man start fires?

What is the soundtrack from the videogame valkyrie of the battlefield: gallian chronicles
valkyrie of the battlefield : gallian
valkyrie of the battlefield: gallian chronicles

what's a track listed on fidos summer fun, volume 2
fidos summer fun , volume
fido's summer fun, volume 2

what genre of music is found on fillmore west 1969: the complete recordings
fillmore west 1969 : the complete
fillmore west 1969: the complete recordings

Which breed of dog has the colors of liver & white
liver &
nan

What is the sweet hereafter?
sweet hereafter
the sweet hereafter

What's an album by tech n9ne
tech n9ne
album

who is the creator of the altered beast game
altered beast game
altered beast

what event occurred in bound brook, new jersey
bound brook
bound brook, new jersey

Where was the big easy filmed?
big easy
the big easy

who wrote the story for the film safety last!
safety l

oconomowoc
oconomowoc, wisconsin

what kind of film is the bushbaby
bushbaby
the bushbabies

which company is tinashé with?
tinashe
tinashé

what netflix genre was the title  pretend you dont see her 
pretend you dont see her
nan

who is the composer of the song hyperactive!?
hyperactive
hyperactive!

what is an active ingredient in hepataplex 3/3/6/12/3/3/12/6/3/12/3 liquid
hepataplex 3/3/6/12/3/3/12/6/3/12/3
hepataplex 3/3/6/12/3/3/12/6/3/12/3 liquid

what medium was used to paint bonaparte crossing the alps?
paint bonaparte crossing the alps
bonaparte crossing the alps

where was the birth place of malin krastev
malin krastev
nan

what is an animated musical film
animated
musical

what's one of the causes of major deforestation
major deforestation
deforestation

what kind of television show is mission: impossible
mission :
mission: impossible

what song is on the recording the empire strickes back - the imperial march
empire strickes back - the imperial
the empire strickes back - th

billy ocean
billie ocean

what company did sponsor the homeward bound: the incredible journey movie
homeward bound : the incredible
homeward bound: the incredible journey

what type of material was used to paint portrait of raymond radiguet 
paint portrait of raymond radiguet
portrait of raymond radiguet

where is  françois baby house contained
francois baby house
françois baby house

What genre of music are the pussycat dolls?
pussycat dolls
the pussycat dolls

What sort of rock does the flames write
flames
the flames

what type of book binding is unter null.
unter null
unter null.

Name a location in washington (u.s. state)
washington ( u.s.
washington

what plays have susan stroman choreographed?
susan stroman choreographed
susan stroman

what episode aired before intervention (buffy the vampire slayer)
intervention ( buffy the vampire
intervention

what compilation album did frank zappa release?
frank zappa
compilation album

Who was the producer for the raven?
raven
the raven

Wha

In [3]:
import psycopg2
import psycopg2.extras

# Load .env file
pass_ = {}
for line in open('../../.pass'):
    split = line.strip().split('=')
    pass_[split[0]] = split[1]

# Connect    
connection = psycopg2.connect(dbname=pass_['DB_NAME'],
                              port=pass_['DB_PORT'],
                              user=pass_['DB_USER'],
                              host=pass_['DB_HOST'],
                              password=pass_['DB_PASS'])
cursor = connection.cursor()

In [29]:
hits = Search.from_dict({
    'size' : 10,
    'query': {
        'nested' : {
            'path' : 'names',
            'score_mode' : 'max',
            'query' : {
                'fuzzy' : {
                    'names.name' : {
                        'value' :         'skyshaper',
                        'fuzziness' :     5
                    }
                }
            }
        }
    }
}).using(client).index('fb5m_entities').execute()
max_score = hits[0].meta.score
print(max_score)
print(hits[0].__dict__)
ret = []
for hit in hits:
    if hit.score == max_score:
        ret.append(hit)
print(hit)

15.693656
{'_d_': {'names': [{'name': 'skyshaper'}], 'facts': [{'property': 'music/release/track_list', 'subjects': ['0mg1nyt']}, {'property': 'music/release/region', 'subjects': ['07ssc']}, {'property': 'music/release/track', 'subjects': ['018yv8q']}], 'mid': '0f47h35'}, 'meta': {'score': 15.693656, 'index': 'fb5m_entities', 'doc_type': '...}}


AttributeError: 'Hit' object has no attribute 'score'

In [24]:
import pprint

from elasticsearch_dsl.connections import connections
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from fuzzywuzzy import fuzz

# Define a default Elasticsearch client
connections.create_connection(hosts=['localhost'])
client = Elasticsearch()

def get_object_link(object_, relation_to_confidence, print_info=False):
    """
    Link the object_ (str) to a QID.
    
    Args:
        object_ (str): object to link
        predicate_mid (str): MID to filter the results
        print_info (bool): print the top matches
    Return:
        mid (str): object QID
        score (float): the score assigned to the top result
        name (str): name of the object QID
    """
    print(object_)
    hits = Search.from_dict({
        'size' : 10,
        'query': {
            'nested' : {
                'path' : 'names',
                'score_mode' : 'max',
                'query' : {
                    'fuzzy' : {
                        'names.name' : {
                            'value' :         object_.lower(),
                            'fuzziness' :     20,
                            'max_expansions': 300
                        }
                    }
                }
            }
        }
    }).using(client).index('fb5m_entities').execute()
    print(hits)
    if len(hits) == 0:
        print('WARNING: No Object found.')
        return None, None, None, None
    return hits[0].mid, hits[0].meta.score, hits[0].names, hits[0].facts

print(object_, relation)
get_object_link(object_, relation, print_info=True)

us route 6 www.freebase.com/transportation/road/major_cities
us route 6
<Response: []>


(None, None, None, None)

In [64]:
import ujson as json
import pandas as pd

from IPython.display import display

SRC = '../../data/simple_qa/dev.tsv'

def answer_question(question):
    """
    Answer a question in JSON format for IO.
    
    Args:
        question (str)
    Returns:
        DL Predicate (str): name of the predicate
        DL Preidcate PID (str): PID of the predicate
        DL Preidcate Confidence (float)
        DL Top Predicates (list of predicate, PID, and confidence): Top predicates about > .9 confience
        DL Object (str): name of the object in the question
        DL Object Confidence (float)
        DL Object Name (str): name of the Wikidata object linked too
        DL Object Aliases (list of str): list of aliases for the object
        DL Object ID (str): QID for th WikiData object
        DL Object Score (float): score by ElasticSearch for object linking
        DL Answers (list of Object ID, Score, Answer): list of tuples that have the PID and QID
    """
    question = preprocess(question)
    print('Question:     ', question)
    top_predicates = get_relation(question, top_k=3)
    print('Relation(s):  ', top_predicates)
    top_mid = None
    top_score = 0
    top_names = None
    top_relation = None
    top_subjects = None
    object_, object_confidence = get_object(question)
    print('Object:       ', object_, '(%.01f)' % object_confidence)
    for relation, confidence in top_predicates:
        mid, score, names, facts = get_object_link(object_, relation)
        print('Object FB:    ', names)
        print('Object Score: ', score)
        print('Relation:     ', relation)
        if mid and score > top_score:
            top_subjects = set()
            for fact in facts:
                if fact['property'] in relation:
                    top_subjects.update(fact['subjects'])
            top_mid = mid
            top_score = score
            top_names = names
            top_relation = relation
    print('Object MID:   ', top_mid)
    print('Object Score: ', top_score)
    print('Object Names: ', top_names)
    print('Top Relation: ', top_relation)
    print('Top Subjects:  ', top_subjects)
    return top_mid, top_relation, top_subjects

def main():
    """
    Run main to save answers to the pandas table SRC
    
    Object Accuracy: 94%
    Object Linking: 75.7%
    Relation Accuracy: 82%
    Relation Accuracy Top 1: 77% ~> 81%
    Relation Accuracy Top 2: 88% ~> 91%
    Accuracy: 72%
    Total Questions: 1514
    """
    data = pd.read_table(SRC)
    display(data.head())
    
    def add_answers(row):
        question = row['Question EN']
        predicted_object, predicted_relation, predicted_subjects = answer_question(question)
        true_object = row['Object MID'].replace('www.freebase.com/m/', '').strip()
        true_subject = row['Subject MID'].replace('www.freebase.com/m/', '').strip()
        true_relation = row['Freebase Property'].strip()
        if not predicted_subjects or true_subject not in predicted_subjects:
            if true_object != predicted_object:
                print('Wrong Object MID:    ', predicted_object)
            print('Correct Object MID:  ', true_object)
            print('Correct Object:      ', row['Object EN'])
            
            if true_relation != predicted_relation:
                print('Wrong Relation:      ', predicted_relation)
            print('Correct Relation:    ', true_relation)
            
            if predicted_subjects and true_subject not in predicted_subjects:
                print('Wrong Subjects:      ', predicted_subjects)
            print('Correct Subject MID: ', true_subject)
            print('WHOOPS!')
        else:
            print('CORRECT!')
        print()
    
    data = data.apply(add_answers, axis=1)
    print('Done!')

main()

,Object MID,Freebase Property,Subject MID,Question,WikiData Property,Subject EN,Object EN,Question FR DeepL,Question EN,Object EN Mask
0,www.freebase.com/m/0f3xg_,www.freebase.com/symbols/namesake/named_after,www.freebase.com/m/0cqt90,Who was the trump ocean club international hot...,P138,Donald Trump,Trump Ocean Club International Hotel and Tower,qui était l'hôtelier et la tour internationale...,Who was the trump ocean club international hot...,c c c e e e e e e e c c
1,www.freebase.com/m/07f3jg,www.freebase.com/people/person/place_of_birth,www.freebase.com/m/0565d,where was sasha vujačić born,P19,Maribor,Sasha Vujačić,où est né sasha vuja?i?,where was sasha vujačić born,c c e e c
2,www.freebase.com/m/031j8nn,www.freebase.com/music/release/region,www.freebase.com/m/07ssc,What is a region that dead combo was released in,NaN,United Kingdom,Dead Combo,quelle est la région dans laquelle le combo mo...,What is a region that dead combo was released in,c c c c c e e c c c
3,www.freebase.com/m/0c1cyhd,www.freebase.com/film/director/film,www.freebase.com/m/0wxsz5y,What is a film directed by wiebke von carolsfeld?,inverse:P57,Stay,Wiebke von Carolsfeld,Qu'est-ce qu'un film réalisé par wiebke von ca...,What is a film directed by wiebke von carolsfeld?,c c c c c c e e e
4,www.freebase.com/m/0fvhc0g,www.freebase.com/music/release/region,www.freebase.com/m/0345h,what country was music for stock exchange rel...,NaN,Germany,Music for Stock Exchange,quel pays était la musique de la bourse publié...,what country was music for stock exchange rel...,c c c e e e e c c


Question:      who was the trump ocean club international hotel and tower named after
Relation(s):   [('www.freebase.com/symbols/namesake/named_after', 0.9895257231903946), ('www.freebase.com/organization/organization/advisors', 0.004211182840038119), ('www.freebase.com/music/recording/artist', 0.0020376382557712215)]
Object:        trump ocean club international hotel and tower (1.0)
Object FB:     [{'name': 'trump ocean club international hotel and tower'}]
Object Score:  31.031027
Relation:      www.freebase.com/symbols/namesake/named_after
Object FB:     [{'name': 'isl international sport and leisure'}]
Object Score:  7.9539623
Relation:      www.freebase.com/organization/organization/advisors
Object FB:     [{'name': 'trump tower'}]
Object Score:  18.09557
Relation:      www.freebase.com/music/recording/artist
Object MID:    0f3xg_
Object Score:  31.031027
Object Names:  [{'name': 'trump ocean club international hotel and tower'}]
Top Relation:  www.freebase.com/symbols/namesake/n

Object FB:     [{'name': 'john hunyadi'}]
Object Score:  13.287171
Relation:      www.freebase.com/people/person/nationality
Object FB:     [{'name': 'family'}, {'name': 'masters of horror : john landis : family'}]
Object Score:  10.036607
Relation:      www.freebase.com/film/film/country
Object MID:    01px0j
Object Score:  20.348745
Object Names:  [{'name': 'hunyadi family'}]
Top Relation:  www.freebase.com/people/family/country
Top Subjects:   {'03gj2'}
CORRECT!

Question:      what major cities does u.s route 2 run through
Relation(s):   [('www.freebase.com/transportation/road/major_cities', 0.9999427811780749), ('www.freebase.com/location/location/containedby', 3.880588867578666e-05), ('www.freebase.com/location/location/contains', 1.4279428861668248e-05)]
Object:        u.s route 2 (1.0)
Object FB:     [{'name': 'u.s. route 2'}, {'name': 'u.s. highway 2'}]
Object Score:  17.76996
Relation:      www.freebase.com/transportation/road/major_cities
Object FB:     [{'name': 'u.s. highw

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Object:        australia (1.0)
Object FB:     [{'name': 'australia'}]
Object Score:  10.5068445
Relation:      www.freebase.com/music/composition/recordings
Object FB:     [{'name': 'australia'}]
Object Score:  10.5068445
Relation:      www.freebase.com/music/composition/language
Object FB:     None
Object Score:  None
Relation:      www.freebase.com/music/compositional_form/compositions
Object MID:    0nm2g6l
Object Score:  10.5068445
Object Names:  [{'name': 'australia'}]
Top Relation:  www.freebase.com/music/composition/recordings
Top Subjects:   {'02prsd4'}
Wrong Object MID:     0nm2g6l
Correct Object MID:   0zvwcn5
Correct Object:       nan
Correct Relation:     www.freebase.com/music/composition/recordings
Wrong Subjects:       {'02prsd4'}
Correct Subject MID:  0fw_qg3
WHOOPS!

Question:      what is the film tempo di uccidere about
Relation(s):   [('www.freebase.com/media_common/netflix_title/netflix_genres', 0.4808569935592784), ('www.freebase.com/film/film/subjects', 0.3650900

Object:        thriller (1.0)
Object FB:     [{'name': 'thriller'}]
Object Score:  14.047661
Relation:      www.freebase.com/media_common/netflix_genre/titles
Object FB:     [{'name': 'thriller'}]
Object Score:  14.047661
Relation:      www.freebase.com/film/film_genre/films_in_this_genre
Object FB:     [{'name': 'thriller'}]
Object Score:  14.047661
Relation:      www.freebase.com/tv/tv_genre/programs
Object MID:    01jfsb
Object Score:  14.047661
Object Names:  [{'name': 'thriller'}]
Top Relation:  www.freebase.com/media_common/netflix_genre/titles
Top Subjects:   {'04j2dcg', '0crryp5', '04j1pp5', '0c1n5_z', '04j23yr', '02qpnmm', '030xw6', '04nvmwz', '07k482', '04j33zr', '069m26', '04j3446', '0414gmx', '0crsf9v', '0gy4mh', '02r8_xn', '033qdy', '0gm19g4', '04j1pt9', '05b5x97', '03m9t55', '0crt42p', '091w1s', '04q0svn', '089d0v', '0b6_znh', '02r34wj', '0crx42h', '04j0jxz', '0gyy81x', '096nbp', '0crxx7t', '04j2cx5', '0489v9', '0bd5t90', '07k68tf', '0crvk6q', '0cs5nmx', '0f7tms', '0g9tdd

KeyboardInterrupt: 